# Asynchronous programming with Python
## Module 2

### Agenda:

* Evolution of asynchronous frameworks.
* Meet Trio, a friendly Python library for async concurrency and I/O.
* Some practice.

## Evolution of asynchronous frameworks

Long, long time ago...

🔵 ◀︎ 2001 - [Deffered](https://twistedmatrix.com/documents/16.2.0/core/howto/defer.html#visual-explanation) object is introduced with Twisted library.  
⼁  
🔵 ◀︎ 2003 - [PEP-255](https://www.python.org/dev/peps/pep-0255): add `yeld` keyword.  Twisted gets generator-based coroutines.  
⼁  
⼁  
🔵 ◀︎ 2006 - [PEP-342](https://www.python.org/dev/peps/pep-342) add `send` and `throw` methods to generator.  Coroutines made simpler.  
⼁  
⼁  
⼁  
⼁  
⼁  
🔵 ◀︎ 2012 - [PEP-380](https://www.python.org/dev/peps/pep-380) add `yield from` construction.  Coroutines performance is improved.  
⼁  
🔵 ◀︎ 2014 - [PEP-3156](https://www.python.org/dev/peps/pep-3156) add `asyncio` to Python 3.4.  Coroutines are offitially a part of the standard library.  
🔵 ◀︎ 2015 - [PEP-492](https://www.python.org/dev/peps/pep-492) add `async` and `await` syntax.  Coroutines are more readable.  
🔵 ◀︎ 2016 - More syntax updates: asynchronous generators, asynchronous comprehensions.  
⼁  
⼁  
⼁  
⼁  
…

20 years after introducing `Deffered`s, asyncio still uses [Deferred-like objects](https://docs.python.org/3/library/asyncio-task.html#awaitables) to run.

---
***And Now for Something Completely Different***

…  
⼁  
🟣 ◀︎ 2015 - [David Beazley](https://www.dabeaz.com/) introduced [Curio](https://github.com/dabeaz/curio), the next-level\*, coroutine-only asynchronous library.  
⼁  
🟣 ◀︎ 2017 - [Nathaniel Smith](https://vorpus.org/) brought ideas from `Curio` to his new [Trio](https://trio.readthedocs.io/en/stable/index.html) library, implementing structured concurrency\*\*.  
⼁  
…


---

\* From the description of Curio:

Curio strictly separates asynchronous code from synchronous code. Specifically, *all* functionality related to the asynchronous environment utilizes "async/await" features and syntax--without exception. Moreover, interactions between async and sync code is carefully managed through a small set of simple mechanisms such as events and queues. As a result, Curio is small, fast, and significantly easier to reason about.

<div align="right">
    – <a href="https://github.com/dabeaz/curio#curio-is-differen">https://github.com/dabeaz/curio#curio-is-differen</a>
</div>


\*\* **Structured concurrency** is a programming paradigm aimed at improving the clarity, quality, and development time of a computer program by using a structured approach to concurrent programming. The core concept is the encapsulation of concurrent threads of execution (here encompassing kernel and userland threads and processes) by way of control flow constructs that have clear entry and exit points and that ensure all spawned threads have completed before exit.

<div align="right">
    – <a href="https://en.wikipedia.org/wiki/Concurrent_computing">Wikipedia / Concurrent computing </a>
</div>


## Meet Trio

The examples below are taken from [Trio's documentation](https://trio.readthedocs.io/en/stable/).

You may also check out the [official tutorial](https://trio.readthedocs.io/en/stable/tutorial.html), it's nice.

First, install [trio](https://trio.readthedocs.io/en/stable/)
and [asks](https://asks.readthedocs.io/en/latest/index.html)

In [22]:
!pip install trio asks

### Sample program
Trio programs are like

![piping async code with Trio](../images/piping-async-code-trio.png)

In [23]:
import trio

async def double_sleep(x):
    print("started! sleeping now...")
    await trio.sleep(2 * x)
    print("exiting!")

trio.run(double_sleep, 3)  # does nothing for 6 seconds then returns

started! sleeping now...
exiting!


### Nursery
If you want to spawn child tasks, you should be a responsible parent.

In [24]:
import trio

async def child1():
    print("  child1: started! sleeping now...")
    await trio.sleep(1)
    print("  child1: exiting!")


async def child2():
    print("  child2: started! sleeping now...")
    await trio.sleep(2)
    print("  child2: exiting!")


async def parent():
    print("parent: started!")
    async with trio.open_nursery() as nursery:
        print("parent: spawning child1...")
        nursery.start_soon(child1)

        print("parent: spawning child2...")
        nursery.start_soon(child2)

        print("parent: waiting for children to finish...")
        # -- we exit the nursery block here --
    print("parent: all done!")


trio.run(parent)

parent: started!
parent: spawning child1...
parent: spawning child2...
parent: waiting for children to finish...
  child1: started! sleeping now...
  child2: started! sleeping now...
  child1: exiting!
  child2: exiting!
parent: all done!


The children *never* leave the nursery scope, and this is awesome.

![baby ducks under their mother](../images/baby-ducks.png)

---

### Tasks communication
Use [`trio.open_memory_channel`](https://trio.readthedocs.io/en/stable/reference-core.html#using-channels-to-pass-values-between-tasks)
to create a pair of send and receive channels.

In [33]:
import trio


async def main():
    async with trio.open_nursery() as nursery:
        # Open a channel:
        send_channel, receive_channel = trio.open_memory_channel(0)
        # Start a producer and a consumer, passing one end of the channel to
        # each of them:
        nursery.start_soon(producer, send_channel)
        nursery.start_soon(consumer, receive_channel)


async def producer(send_channel):
    async with send_channel:
        # Producer sends 3 messages
        for i in range(3):
            # The producer sends using 'await send_channel.send(...)'
            print(f"sending a message: {i}")
            await send_channel.send(f"message {i}")


async def consumer(receive_channel):
    async with receive_channel:
        # The consumer uses an 'async for' loop to receive the values:
        async for value in receive_channel:
            print(f"got value {value!r}")


trio.run(main)

sending a message: 0
got value 'message 0'
sending a message: 1
sending a message: 2
got value 'message 1'
got value 'message 2'


##### Some practice

**1. Update the code above to have two children processing the messages concurrently.
Pass some ID to the `consumer` function to be able to
tell which child picked which message.**

**2. What happens if you have two producers as well?
Chances are you will occasionally get some errors.**

**2.1. Fix the errors by passing `send_channel.clone()` to producers.**

**3. Pass `receive_channel.clone()` to the children to have
fan-out behaviour.**

👉 *In general you should use unbuffered memory channels by passing `0` to `trio.open_memory_channel`.
Buffer should be defined only if it is required to unblock the producer.
For details see the [Trio documentation](https://trio.readthedocs.io/en/stable/reference-core.html#buffering-in-channels).*👈

### Adwanced topic: cancellation scope

#### A simple timeout example.
Try to run the code below multiple times.

In [26]:
from random import randint

import asks
import trio

async def main():
    with trio.move_on_after(5):
        result = await asks.get(f"https://httpbin.org/delay/{ randint(1, 10) }")
        print("result is", result)
    print("with block finished")


trio.run(main)


with block finished


#### Nested scopes
How do you think, what the code below will print out?

In [27]:
import trio

async def main():
    print("starting...")
    with trio.move_on_after(2):
        with trio.move_on_after(4):
            await trio.sleep(5)
            print("sleep finished without error")
        print("move_on_after(4) finished without error")
    print("move_on_after(2) finished without error")

trio.run(main)

starting...
move_on_after(2) finished without error


##### Some practice

**1. Change the values passed to `move_on_after` so that the inner one takes effect.**

👉 *You can get `trio.CancelScope` instance using
`with trio.move_on_after(...) as cancel_scope:`.
Setting `cleanup_scope.shield = True` shields the cancel scope from
the upstream cancellation.
Setting `cleanup_scope.deadline += N` increases the deadline by N seconds.*👈

**2. Instead of changing the values of `move_on_after`,
update one of the cancellation scope instances so that the
inner one takes effect.**


#### Shielding
How do you think, what the code below will print out?

In [28]:
import trio

async def main():
    with trio.move_on_after(2):
        # This request blocks for 4 seconds before returning
        try:
            print("Starting a blocking request.")
            await asks.get(f"https://httpbin.org/delay/3")
            print("The first request succeeded.")
        except trio.Cancelled:
            print("The first request timed out, try with a smaller delay.")
            await asks.get(f"https://httpbin.org/delay/1")
            print("The second request succeeded.")

    print("Done.")


trio.run(main)

Starting a blocking request.
The first request timed out, try with a smaller delay.
Done.


##### Some practice

**Update the code so that the second request succeeds in case the
first one failed.**

### Testing
For testing Trio with `pytest` use
[pytest-trio](https://pytest-trio.readthedocs.io/en/stable/index.html).

First, install the test libraries.

In [1]:
!pip install pytest pytest-trio ipytest

In [8]:
# Some magic to enable tests in the notebook
import ipytest
ipytest.autoconfig(clean=True, addopts=("--capture=tee-sys", "-qq"))

#### Creating a test

In [9]:
%%run_pytest
import pytest
import trio


async def double_sleep(x):
    sleep_for = 2 * x
    print("started! sleeping now...")
    await trio.sleep(sleep_for)
    print("exiting!")
    return sleep_for


@pytest.mark.trio
async def test_double_sleep():
    assert await double_sleep(3) == 6

started! sleeping now...
exiting!
.                                                                        [100%]


👉 *If you don't want to mark each test with `@pytest.mark.trio`, there
are other ways to enable Trio mode globally,
see [the documentation](https://pytest-trio.readthedocs.io/en/stable/reference.html#trio-mode).*👈

#### Mock clocks
The test above can be speed up with an
[autojump_clock](https://pytest-trio.readthedocs.io/en/stable/reference.html#built-in-fixtures)
fixture.

In [10]:
%%run_pytest
@pytest.mark.trio
async def test_double_sleep(autojump_clock):
    assert await double_sleep(3) == 6
    assert trio.current_time() == 6

started! sleeping now...
exiting!
.                                                                        [100%]


👉 *If you need more control on the clock skews, you can use
a [mock_clock](https://pytest-trio.readthedocs.io/en/stable/reference.html#built-in-fixtures).*👈

#### Checking checkpoints
You should always ensure that your code have enough Trio checkpoints
so that context switching and cancellation mechanizm works.

Consider this example.

In [14]:
%%run_pytest

async def broken_double_sleep(x):
    sleep_for = 2 * x
    print("started! sleeping now...")

    if sleep_for % 2:
        # This will work only if x is a fractional number
        await trio.sleep(sleep_for)

    print("exiting!")
    return sleep_for


@pytest.mark.trio
async def test_broken_double_sleep():
    assert await broken_double_sleep(3) == 6

started! sleeping now...
exiting!
.started! sleeping now...
exiting!
F                                                                       [100%]
=================================== FAILURES ===================================
___________________________ test_broken_double_sleep ___________________________

value = <trio.Nursery object at 0x10cf677d0>

    async def yield_(value=None):
>       return await _yield_(value)

../.venv/lib/python3.7/site-packages/async_generator/_impl.py:106: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
../.venv/lib/python3.7/site-packages/async_generator/_impl.py:99: in _yield_
    return (yield _wrap(value))
<ipython-input-14-971b9d2388f9>:16: in test_broken_double_sleep
    assert await broken_double_sleep(3) == 6
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

self = <contextlib._GeneratorContextManager object at 0x10cf67590>, type = None
value = None, traceback = None

    def __e

The test passes.  Now, try wrapping the last line with `with trio.testing.assert_checkpoints():`.

#### References
For more useful test utilities refer to
[pytest-trio](https://pytest-trio.readthedocs.io/en/stable/index.html)
and [trio.testing](https://trio.readthedocs.io/en/stable/reference-testing.html#module-trio.testing)
documentation.

## Conclusion
Trio helps to write code with a focus on usability and correctness.
It makes it a great framework to create your first asynchronous
application.

The downside of Trio is that it is not as "mainstream" as
`asyncio`.  Even though Trio ecosystem is robust, you may find it
hard to get libraries for the cases where `asyncio` is already used.